In [35]:
# converted to python from mathematica sample
# using chatgpt and many manual changes 12/17/2022
import random
import pandas as pd
import matplotlib.pyplot as plt
def weighted_coin_toss_2(pay_out_odds, chance_of_winning):
    if random.random() > 1 - chance_of_winning:
        return pay_out_odds - 1
    else:
        return -1
    
def RunGame2(wlt, OriginalAmountOfMoney, BettingStyle, 
             ChanceOfWinning, FractionBetting, CanGoIntoDebt, BettingLimit, MaxBet, PayOutOdds, ConstantBet):
    Monies = [OriginalAmountOfMoney, OriginalAmountOfMoney]
    for i in range(len(wlt)):
        newMonies = Monies[-1] + wlt[i] * Bet2(Monies, BettingStyle, ChanceOfWinning, FractionBetting, CanGoIntoDebt, BettingLimit, MaxBet, PayOutOdds, ConstantBet)
        #print(newMonies)
        Monies.append(newMonies)
    return Monies

def Bet2(winnings_history, style, chance_of_winning, fraction_betting, can_go_into_debt, betting_limit, max_bet, pay_out_odds, constant_bet):
    current_winnings = winnings_history[-1]
    kbet = 0 if pay_out_odds == 1 else ((pay_out_odds * chance_of_winning - 1) * current_winnings) / (pay_out_odds - 1)
  
    fbet = fraction_betting * current_winnings
    mbet = abs(winnings_history[-1] - winnings_history[-2])
    
    if style == "constant bet":
        if current_winnings >= constant_bet:
            return constant_bet
        elif can_go_into_debt:
            return constant_bet
        else:
            return 0
    elif style == "Kelly criterion":
        if kbet > 0:
            print(f"chance of winning:{chance_of_winning}, currentwinnnings: {current_winnings},kbet {kbet},percentage of winnings {kbet/current_winnings}")    
            if betting_limit:
                
                return min(kbet, max_bet)
            else:
                
                return kbet
        else:
            return 0
    elif style == "constant fraction":
        if betting_limit:
            return min(fbet, max_bet)
        else:
            return fbet
    elif style == "martingale":
        if len(winnings_history) == 2:
            return 1
        elif winnings_history[-1] - winnings_history[-2] > 0:
            return mbet
        else:
            return min(2 * mbet, max_bet if betting_limit else float("inf"), float("inf") if can_go_into_debt else current_winnings)
    elif style == "anti-martingale":
        if len(winnings_history) == 2:
            return 1
        elif winnings_history[-1] - winnings_history[-2] > 0:
            return min(2 * mbet, max_bet if betting_limit else float("inf"), float("inf") if can_go_into_debt else current_winnings)
        else:
            return min(mbet / 2, max_bet if betting_limit else float("inf"), float("inf") if can_go_into_debt else current_winnings)
    else:
        return 1

def GetData2(ChanceOfWinning, NumGames, OriginalAmountOfMoney, FractionBetting, 
             CanGoIntoDebt, BettingLimit, MaxBet, seed, PayOutOdds, ConstantBet):
    wlt = []
    if seed:
        random.seed(seed)
    for _ in range(NumGames):
        if random.random() > 1 - ChanceOfWinning:
            wlt.append(PayOutOdds - 1)
        else:
            wlt.append(-1)
    y1 = RunGame2(wlt, OriginalAmountOfMoney, "constant bet", ChanceOfWinning, .1, CanGoIntoDebt, BettingLimit, MaxBet, PayOutOdds, ConstantBet)
    y2 = RunGame2(wlt, OriginalAmountOfMoney, "Kelly criterion", ChanceOfWinning, .1, CanGoIntoDebt, BettingLimit, MaxBet, PayOutOdds, 0)
    #print("kbet",y2)
    y3 = RunGame2(wlt, OriginalAmountOfMoney, "constant fraction", ChanceOfWinning, FractionBetting, CanGoIntoDebt, BettingLimit, MaxBet, PayOutOdds, 0)
    y4 = RunGame2(wlt, OriginalAmountOfMoney, "martingale", ChanceOfWinning, .1, CanGoIntoDebt, BettingLimit, MaxBet, PayOutOdds, 0)
    y5 = RunGame2(wlt, OriginalAmountOfMoney, "anti-martingale", ChanceOfWinning, .1, CanGoIntoDebt, BettingLimit, MaxBet, PayOutOdds, 0)
    return y1, y2, y3, y4, y5




In [36]:
import matplotlib.pyplot as plt



def Chart(ChanceOfWinning, NumGames, OriginalAmountOfMoney, 
          FractionBetting, CanGoIntoDebt, BettingLimit, MaxBet, 
          seed,
          PayOutOdds, ConstantBet):
    sum1, sum2, sum3, sum4, sum5 = 0, 0, 0, 0, 0
    for _ in range(25):
        data = GetData2(ChanceOfWinning, NumGames, OriginalAmountOfMoney, 
                        FractionBetting, CanGoIntoDebt, BettingLimit, MaxBet, 
                        seed,
                        PayOutOdds, ConstantBet)
        
        sum1 += data[0][-1] 
        sum2 += data[1][-1] 
        sum3 += data[2][-1] 
        sum4 += data[3][-1] 
        sum5 += data[4][-1] 
    return sum1, sum2, sum3, sum4, sum5

def onelegend(color_text):
    color, text = color_text
    fig, ax = plt.subplots()
    ax.plot([], [], color=color, linewidth=10)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.legend([text])
    plt.show()

In [37]:
fig = plt.figure(figsize=(1000,500))
OriginalAmountOfMoney = 1000
for winChance in range(10,70,10):
    df = pd.DataFrame(columns=["ContantBet", "Kelly", "ContantFraction","Martingale","AntiMartingale"])
    for i in range(10):
        values = Chart(ChanceOfWinning=winChance/100.0, NumGames=100, OriginalAmountOfMoney=OriginalAmountOfMoney, 
              FractionBetting=0.02, CanGoIntoDebt=True, BettingLimit=10, MaxBet=OriginalAmountOfMoney, 
              seed=None,
              PayOutOdds=2, ConstantBet=10)
        
        avg_values = tuple(item / 25 for item in values)
        df.loc[i] = avg_values
    df[["ContantBet", "Kelly", "ContantFraction"]].plot(title="WinChance {} Orignal {}".format(winChance/100,OriginalAmountOfMoney))


chance of winning:0.6, currentwinnnings: 1000,kbet 199.99999999999994,percentage of winnings 0.19999999999999996
chance of winning:0.6, currentwinnnings: 1200.0,kbet 239.99999999999994,percentage of winnings 0.19999999999999996
chance of winning:0.6, currentwinnnings: 1440.0,kbet 287.99999999999994,percentage of winnings 0.19999999999999996
chance of winning:0.6, currentwinnnings: 1152.0,kbet 230.39999999999995,percentage of winnings 0.19999999999999996
chance of winning:0.6, currentwinnnings: 921.6,kbet 184.31999999999996,percentage of winnings 0.19999999999999996
chance of winning:0.6, currentwinnnings: 1105.92,kbet 221.18399999999997,percentage of winnings 0.19999999999999996
chance of winning:0.6, currentwinnnings: 884.7360000000001,kbet 176.94719999999998,percentage of winnings 0.19999999999999996
chance of winning:0.6, currentwinnnings: 1061.6832000000002,kbet 212.33664,percentage of winnings 0.19999999999999996
chance of winning:0.6, currentwinnnings: 1274.0198400000002,kbet 254

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [24]:
df.head()

,ContantBet,Kelly,ContantFraction,Martingale,AntiMartingale
0,1192.8,11363.480486,1468.157765,17033.60,7178.014375
1,1209.6,13731.917484,1518.441012,16459.84,8826.456846
2,1184.0,9843.549418,1433.112207,14812.88,5729.955098
3,1192.8,10862.622961,1474.655489,16997.92,7281.416875
4,1187.2,10133.192003,1462.101164,15581.48,6497.070039
